In [1]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-07-08 11:47:21.737776: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
train_dir = 'data/train'
test_dir = 'data/test'
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_generator = train_datagenerator.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

test_generator = test_datagenerator.flow_from_directory(
        test_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
recognition_model = Sequential()

recognition_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
recognition_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
recognition_model.add(MaxPooling2D(pool_size=(2, 2)))
recognition_model.add(Dropout(0.25))

recognition_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
recognition_model.add(MaxPooling2D(pool_size=(2, 2)))
recognition_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
recognition_model.add(MaxPooling2D(pool_size=(2, 2)))
recognition_model.add(Dropout(0.25))

recognition_model.add(Flatten())
recognition_model.add(Dense(1024, activation='relu'))
recognition_model.add(Dropout(0.5))
recognition_model.add(Dense(7, activation='softmax'))

2022-07-08 11:47:24.394885: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-07-08 11:47:24.438321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.515GHz coreCount: 14 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 178.84GiB/s
2022-07-08 11:47:24.438364: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-07-08 11:47:24.445678: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-07-08 11:47:24.445716: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-07-08 11:47:24.462728: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuf

In [4]:
recognition_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         1

In [5]:
cv2.ocl.setUseOpenCL(False)

In [6]:
recognition_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001, decay=1e-6),metrics=['accuracy'])

In [7]:
recognition_model_info = recognition_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=60,
        validation_data=test_generator,
        validation_steps=7178 // 64)
recognition_model.save_weights('recognition_model.h5')

/home/pkm/.local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1935: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-07-08 11:47:42.020966: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-08 11:47:42.041305: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3293840000 Hz


Epoch 1/60


2022-07-08 11:47:42.377324: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-07-08 11:47:43.118291: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8401
2022-07-08 11:47:43.910616: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-07-08 11:47:43.910887: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


448/448 [==============================] - 12s 22ms/step - loss: 1.7960 - accuracy: 0.2645 - val_loss: 1.7178 - val_accuracy: 0.3421
Epoch 2/60
448/448 [==============================] - 9s 19ms/step - loss: 1.6182 - accuracy: 0.3697 - val_loss: 1.5276 - val_accuracy: 0.4189
Epoch 3/60
448/448 [==============================] - 9s 20ms/step - loss: 1.5135 - accuracy: 0.4185 - val_loss: 1.4447 - val_accuracy: 0.4475
Epoch 4/60
448/448 [==============================] - 9s 20ms/step - loss: 1.4458 - accuracy: 0.4451 - val_loss: 1.3925 - val_accuracy: 0.4707
Epoch 5/60
448/448 [==============================] - 9s 20ms/step - loss: 1.3904 - accuracy: 0.4707 - val_loss: 1.3434 - val_accuracy: 0.4849
Epoch 6/60
448/448 [==============================] - 9s 20ms/step - loss: 1.3386 - accuracy: 0.4921 - val_loss: 1.3039 - val_accuracy: 0.5061
Epoch 7/60
448/448 [==============================] - 9s 20ms/step - loss: 1.3007 - accuracy: 0.5045 - val_loss: 1.2741 - val_accuracy: 0.5121
Epoch 8/6

448/448 [==============================] - 9s 20ms/step - loss: 0.3156 - accuracy: 0.8881 - val_loss: 1.2985 - val_accuracy: 0.6147
Epoch 59/60
448/448 [==============================] - 9s 20ms/step - loss: 0.3065 - accuracy: 0.8902 - val_loss: 1.3042 - val_accuracy: 0.6148
Epoch 60/60
448/448 [==============================] - 9s 20ms/step - loss: 0.3056 - accuracy: 0.8900 - val_loss: 1.2786 - val_accuracy: 0.6176


In [8]:
facial_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [9]:
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('haarcascades_cuda/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        facial_prediction = recognition_model.predict(cropped_img)
        maxindex = int(np.argmax(facial_prediction))
        cv2.putText(frame, facial_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to target thread (0x565525aa4a40)

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to target thread (0x565525aa4a40)

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to target thread (0x565525aa4a40)

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to target thread (0x565525aa4a40)

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to target thread (0x565525aa4a40)

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to target thread (0x565525aa4a40)

QObject::moveToThread: Current thread (0x565525aa4a40) is not the object's thread (0x565525aac820).
Cannot move to tar